<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-11/blob/main/Primer%20modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primer modelo

## Preparación del entorno de trabajo

Lo primero que vamos a hacer es preparar el entorno de trabajo. Para ello, descargaremos e importaremos las librerías necesarias:

Instalamos las librerías necesarias:

In [1]:
!pip install fastai --upgrade -q

     |████████████████████████████████| 204kB 4.1MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 


Las importamos:

In [2]:
from fastai.vision.all import *
import os
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
import shutil

Descargamos las imágenes:

In [17]:
!wget https://drive.google.com/file/d/1WS7kotw0iuFxMpzGkiaNVWm8SLr_uNT4/view?usp=sharing -O datos.csv

--2021-05-21 19:37:12--  https://drive.google.com/file/d/1WS7kotw0iuFxMpzGkiaNVWm8SLr_uNT4/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 64.233.170.102, 64.233.170.139, 64.233.170.100, ...
Connecting to drive.google.com (drive.google.com)|64.233.170.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘datos.csv’

datos.csv               [ <=>                ]  71.26K  --.-KB/s    in 0.004s  

2021-05-21 19:37:12 (19.4 MB/s) - ‘datos.csv’ saved [72975]



In [ ]:
!unzip datos.zip

Una vez tenemos las imágenes, debemos descargar las etiquetas. Cada etiqueta se relaciona con su imagen correspondiente por un ID (el nombre de la imagen). Estas etiquetas son necesarias para poder entrenar nuestro modelo.

In [3]:
!wget https://www.dropbox.com/s/acezkcl7d1wfevz/LaRiojaEnLaMemoria.csv?dl=1 -O data.csv -q

Importamos los datos del CSV a un dataframe (usando pandas) para poder tratarlos.

In [4]:
df = pd.read_csv("data.csv")
df.head()

,ID_NUM,IDENT,TITULO,TITULAR,DESCRIPCION,PALABRAS_CLAVE,ANNO,LUGAR,LOCALIDAD,PROVINCIA,PAIS,PROVEEDOR
0,80,LRM-2008/00014-Imagen,Foto con los abuelos,Fotografía de los abuelos con la nieta de dos años en Hormilleja,NaN,NaN,1961,NaN,Hormilleja,NaN,NaN,NaN
1,81,LRM-2008/00015-Imagen,Madre e hijos en el Espolón,Madre e hijos en el Espolón,NaN,NaN,1961,NaN,Logroño,NaN,NaN,NaN
2,82,LRM-2008/00016-Imagen,SrTraspaderne,Severiano Traspaderne con su furgoneta Citroen 2CV de reparto de piensos,NaN,NaN,1964,NaN,Logroño,NaN,NaN,NaN
3,83,LRM-2008/00017-Imagen,Invierno en moto,Sr Aragón y Sr García pertrechados para soportar el invierno en su motol (Rieju 125),NaN,NaN,1964,NaN,Logroño,NaN,NaN,NaN
4,84,LRM-2008/00018-Imagen,Empujando el microcoche,Empujando un BMW Isetta,NaN,NaN,1962,NaN,Logroño,NaN,NaN,NaN


Ahora, creamos una columna adicional que nos permita clasificar las imágenes según su decada:

In [5]:
años = df['ANNO'].values
decadas = []
for año in años:
  decadas.append(str(año)[:3] + '0')
df['DECADA'] = decadas
df.head()

,ID_NUM,IDENT,TITULO,TITULAR,DESCRIPCION,PALABRAS_CLAVE,ANNO,LUGAR,LOCALIDAD,PROVINCIA,PAIS,PROVEEDOR,DECADA
0,80,LRM-2008/00014-Imagen,Foto con los abuelos,Fotografía de los abuelos con la nieta de dos años en Hormilleja,NaN,NaN,1961,NaN,Hormilleja,NaN,NaN,NaN,1960
1,81,LRM-2008/00015-Imagen,Madre e hijos en el Espolón,Madre e hijos en el Espolón,NaN,NaN,1961,NaN,Logroño,NaN,NaN,NaN,1960
2,82,LRM-2008/00016-Imagen,SrTraspaderne,Severiano Traspaderne con su furgoneta Citroen 2CV de reparto de piensos,NaN,NaN,1964,NaN,Logroño,NaN,NaN,NaN,1960
3,83,LRM-2008/00017-Imagen,Invierno en moto,Sr Aragón y Sr García pertrechados para soportar el invierno en su motol (Rieju 125),NaN,NaN,1964,NaN,Logroño,NaN,NaN,NaN,1960
4,84,LRM-2008/00018-Imagen,Empujando el microcoche,Empujando un BMW Isetta,NaN,NaN,1962,NaN,Logroño,NaN,NaN,NaN,1960


Finalmente, etiquetamos las imágenes a partir de la información de los metadatos.

Para ello, vamos a obtener las diferentes décadas en las que están tomadas las imágenes:

In [6]:
decades = set(df['DECADA'])
print(decades)

{'1900', '1910', '1880', '1860', '1970', '1890', '1870', '1960', '1940', '1930', '1990', '1980', '1920', '1950'}


Creamos las carpetas contenedoras. El nombre de la carpeta es el año en que la foto está tomada.

In [7]:
path = Path('1')
for decade in decades:
  (path/str(decade)).mkdir(parents=True,exist_ok=True)

Antes de nada, eliminamos las imágenes que no puedan ser tratadas por la librería:

In [ ]:
for image in verify_images(get_image_files(path)):
  os.remove(str(image))

Solo falta meter en cada carpeta las imágenes correspondientes. Para ello, definimos dos funciones:
- getIdFromPath: devuelve el id de la imagen según su path.
- getImageDecadeById(): devuelve la década de la imagen según su id.

In [8]:
def getIdFromPath(path):
  pathAsString = str(path)
  pointIndex = pathAsString.index(".")
  pathAsString = pathAsString[2:pointIndex]
  return pathAsString

In [9]:
def getImageDecadeById(id):
  try:
    return df[df['ID_NUM'] == int(id)]['DECADA'].values[0]
  except:
    return -1

Para cada imagen, obtenemos su id, buscamos su década en la tabla y la movemos a la carpeta del año a la que pertenece.

Si la imagen no tiene un año asignado o no está incluida en los metadatos, se elimina, ya que no podemos utilizarla para entrenar el modelo.

In [ ]:
images = get_image_files(path)

for image in images:
  imageId = getIdFromPath(image)
  imageYear = getImageYearById(imageId)
  if imageYear != -1:
    shutil.move(str(image),path/(str(imageYear)+'/'+image.name))
  else:
    print("No data for image " + str(id))
    os.remove(image)

Como se puede observar, hay algunas imágenes que no están recogidas en la tabla, por lo que no las podremos usar.

Finalmente, eliminamos los archivos que ya no vamos a utilizar:

In [ ]:
os.remove('data.csv')
os.remove('datos.zip')

## Separación en entrenamiento y test

Una vez que tenemos las imágenes separadas por décadas, podemos diferenciar el conjunto de entrenamiento y el de test. Lo hemos hecho en este orden para asegurar que hay suficientes imágenes en cada categoría.

In [ ]:
finalPath = Path('data')
trainPath = Path('data/train')
testPath = Path('data/test')

for decade in decades:
  trainDecade, testDecade = train_test_split(get_image_files(path/(str(decade))),test_size=0.2,random_state=15)

  for x in trainDecade:
    shutil.move(str(x),finalPath/('train/' + str(decade) + '/'+x.name))

  for x in testDecade:
    shutil.move(str(x),finalPath/('test/' + str(year) + '/'+x.name))

Ya podemos eliminar la carpeta inicial:

In [ ]:
shutil.rmtree('1')

## Definición del DataBlock y el DataLoader

A continuación definiremos estos dos objetos necesarios para poder entrenar el modelo.

### DataBlock

El DataBlock es el objeto que, gracias a una serie de opciones que se especifican en su constructor, permite cargar las imágenes en el modelo mediante un DataLoader.



In [ ]:
dataBlock = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(valid_pct=0.2,seed=42),
                 get_y=parent_label,
                 item_tfms = Resize(512),
                 batch_tfms=aug_transforms(size=256,min_scale=0.75))

### DataLoader

A partir del DataBlock, construimos el DataLoader. Es importante asignar un batch que sea una potencia de 2 para optimizar el uso de la GPU.

In [ ]:
dataLoader = dataBlock.dataloaders(trainPath,bs=128)

Ahora podemos visualizar un fragmento de nuestro dataset, así podemos comprobar que todo ha funcionado correctamente.

In [ ]:
dataLoader.show_batch()

## Creación del modelo

Tal y como se explicó en el entregable 2, vamos a construir una red convolucional utilizando *transfer learning* con arquitectura ResNet 34.

In [ ]:
learn = cnn_learner(dataLoader,resnet18,metrics=accuracy).to_fp16()

### Entrenamiento de la red

Utilizaremos la funcionalidad de fine tune disponible en FastAI.

In [ ]:
learn.fine_tune(10,base_lr=1e-3)

Lo exportamos para uso posterior:

In [ ]:
learn.export()

## Interpretación de los resultados

Creamos una interpretación para poder ver los resultados

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

Podemos visualizar la matriz de confusión, para ver cuántas veces ha acertado nuestro modelo.

In [ ]:
interp.plot_confusion_matrix()

### Evaluación en el conjunto de test

Tenemos que crear nuevos objetos DataBlock Y DataLoader. El primero lo construiremos con un splitter concreto que nos permite especificar la base de datos de test

In [ ]:
dataBlockTest = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=GrandparentSplitter(valid_name='test'),
                 get_y=parent_label,
                 item_tfms = Resize(256),
                 batch_tfms=aug_transforms(size=128,min_scale=0.75))
dataLoaderTest = dataBlockTest.dataloaders(path,bs=128)

Modificamos el anterior learner:

In [ ]:
learn.dls = dataLoaderTest

Evaluamos el modelo mediante el método validate().

In [ ]:
learn.validate()